In [1]:
from DrissionPage import ChromiumPage
import pyautogui
import pandas as pd
import time
import os
from DrissionPage.errors import NoRectError

from DrissionPage import ChromiumPage   #浏览器控制包
import os                                #控制系统文件夹（建议使用管理者模式打开）
from DrissionPage.common import ActionChains   #控制键盘、鼠标的操作，如ctrl，alt，左键，鼠标移动等
from DrissionPage.common import Keys          #控制键盘、鼠标的操作，如ctrl，alt，左键，鼠标移动等
import ddddocr
import imageio
import time
import base64  
from DrissionPage.errors import ElementNotFoundError
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
import re

In [2]:
df = pd.DataFrame(columns=[ '公司名称', '网址名称', '核查意见','是否适用',"其他数据"])

def 文件创建(company_name_list):
    # 遍历PD_name_list  
    for i in company_name_list:
        # 在文件夹'dada'中创建以pd_name命名的文件夹  
        if not os.path.exists('data/'+ i):
            os.makedirs('data/'+ i)  
        else: 
            print('文件夹已存在: '+ i)

def save_screenshot_to_custom_path(custom_path): 
    """
    custom_path:截图保存的地址
    """
    # 获取屏幕截图  
    time.sleep(0.5)
    screenshot = pyautogui.screenshot()  
    # 将截图保存到自定义路径  
    screenshot.save(custom_path)

def ad_to_df(company_name, url_name, inspection_opinion, applicability,other_data):  
    """  
    将数据保存到DF表中。  
  
    参数: 
    company_name (str): 公司名称。  
    url_name (str): 网址名称。  
    inspection_opinion (str): 核查意见。  
    applicability (str): 是否适用。  
    other_data(str):存储其他信息，方便后续排查
  
    返回:  
    None  
    """  
    global df  
    # 将新的数据添加到DataFrame中  
    new_row = { '公司名称': company_name, '网址名称': url_name, '核查意见': inspection_opinion, '是否适用': applicability,'其他数据':other_data}  
    df = df.append(new_row, ignore_index=True)  
    # 将DataFrame保存到Excel文件  
    #df.to_excel('data.xlsx', index=False)

In [3]:
#验证码模块，目前可以处理普通文字输入验证码+滑块验证码（主要处理17）
def dd_验证码识别_gif(img):
    gif_分割(img)
    #ocr = ddddocr.DdddOcr(old=True)  
    ocr = ddddocr.DdddOcr(old=True,show_ad=False)  
    result_counts = defaultdict(int)  
    for i in range(24):
        with open(r"D:\Awork\work\Htzq\RPAv2\frames\frame_"+str(i)+".png", 'rb') as f:  
            image = f.read()  
        res = ocr.classification(image)  
        #print(res)
        if len(res) == 4:  
            result_counts[res] += 1  
    res_max = max(result_counts, key=result_counts.get)  
    #print("Most frequent result:", res_max)
    return res_max

def dd_验证码识别_pt():
    ocr = ddddocr.DdddOcr(old=True,show_ad=False)  
    result_counts = defaultdict(int)  
    with open(r"img/src.jpg", 'rb') as f:
        image = f.read()  
    res = ocr.classification(image)  
    return res

def gif_分割(img_gif):
    gif_path = img_gif
    with imageio.get_reader(gif_path) as reader:
        for i, frame in enumerate(reader):
            frame_path = os.path.join("frames", f"frame_{i}.png")
            imageio.imwrite(frame_path, frame)
            #print(frame_path)

def ddddocr_滑块(big_pic_path,small_pic_path):
    det = ddddocr.DdddOcr(det=False, ocr=False)
    with open(big_pic_path, 'rb') as f:
            target_bytes = f.read()
    with open(small_pic_path, 'rb') as f:
        background_bytes = f.read()
    res = det.slide_match(target_bytes, background_bytes, simple_target=True)
    print(res)
    return res["target"][0]*1.40

In [4]:
def 中华人民共和国国家发展和改革委员会网站_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="中华人民共和国国家发展和改革委员会网站"
    # 指定公司名称  
    page.get("https://www.ndrc.gov.cn/")  
    # 输入公司名称并按回车键  
    page.ele("#qt").input(company_name+"\n")
    # 切换到最新打开的标签页  
    page.to_tab(page.latest_tab)
    # 查找页面显示搜索信息页面的元素的大小，无内容为（1190，234）
    ele1 = page.ele("css:body > div.contentbox.box.clearfix")
    while True:
        if ele1.size[1] == 60:
            continue
        else:
            break
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if ele1.size[1] == 234:
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:  
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是234，就是有搜索内容，搜索的内容保存到df表中。
        ele_text = page.ele("#results").text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")

def 中华人民共和国应急管理部网站_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="中华人民共和国应急管理部网站"
    # 指定公司名称  "
    page.get("http://www.mem.gov.cn")  
    # 输入公司名称并按回车键  
    page.ele("#iptSword").input(company_name+"\n")
    # 切换到最新打开的标签页  
    page.to_tab(page.latest_tab)
    # 查找页面显示搜索信息页面的元素的大小，无内容为（1190，234）
    ele1 = page.ele(".lmdhd")
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if ele1.text[-3:-1] == "共0":
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:  
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是500，就是有搜索内容，搜索的内容保存到df表中。
        ele_text = ele1.text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")
    
    
def 中华人民共和国生态环境部网站_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="中华人民共和国生态环境部网站"
    # 指定公司名称  
    page.get("http://www.mee.gov.cn")  
    # 输入公司名称并按回车键  
    page.ele("#searchword").input(company_name+"\n")
    # 切换到最新打开的标签页  
    page.to_tab(page.latest_tab)
    # 查找页面显示搜索信息页面的元素的大小，无内容为"符合条件的有0条"
    ele1 = page.ele("符合条件的有")
    while True:
        if len(ele1.text) == 7:
            continue
        else:
            break
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if ele1.text == "符合条件的有0条":
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:  
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
        ele_text = page.ele("css:#list > div.ll_gjjs_list.ll_gjjs_list_was").text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")
    
def 中华人民共和国工业和信息化部网站_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="中华人民共和国工业和信息化部网站"
    # 指定公司名称  
    page.get("http://www.miit.gov.cn")  
    # 输入公司名称并按回车键  
    page.ele("#q").input(company_name+"\n")
    # 切换到最新打开的标签页  
    page.to_tab(page.latest_tab)
    # 查找页面显示搜索信息页面的元素的大小，无内容为"符合条件的有0条"
    ele1 = page.ele("为您找到相关结果约")
    while True:
        if len(ele1.text) == 10:
            #print(1)
            continue
        else:
            break
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if ele1.text == "为您找到相关结果约0个":
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:  
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
        ele_text = page.ele("css:#search-form > div.main > div.page-con > div.content > div > div.con_left > div.yyfw > div.zffw_list > div").text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")
    

def 中华人民共和国商务部网站_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="中华人民共和国商务部网站"
    # 指定公司名称  
    page.get("http://www.mofcom.gov.cn/")  
    # 输入公司名称并按回车键  
    page.ele("#searchValue").input(company_name+"\n")
    # 切换到最新打开的标签页  
    page.to_tab(page.latest_tab)
    # 查找页面显示搜索信息页面的元素的大小，无内容为"符合条件的有0条"
    ele1 = page.ele("css:body > section > section > div.row.wms-con > div > div.f-mt50.s-info-box > ul")
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if len(ele1.text) == 33:
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:  
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
        ele_text = ele1.text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")
    
def 中华人民共和国财政部网站_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="中华人民共和国财政部网站"
    # 指定公司名称  
    page.get("http://www.mof.gov.cn")  
    # 输入公司名称并按回车键  
    page.ele("#andsen").input(f"{company_name}\n")
    page.wait.new_tab()
    # 切换到最新打开的标签页  
    page.to_tab(page.latest_tab)
    while True:
        ele2 = page.ele("仍然发送",timeout=0.3)
        if ele2:
            page.ele("仍然发送").click(None)
            continue
        if not ele2:
            break
    # 查找页面显示搜索信息页面的元素的大小，无内容为"符合条件的有0条"
    ele1 = page.ele("css:body > div.mainboxerji > div.list_search")
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if len(ele1.text) == 97:
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:  
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
        ele_text = ele1.text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")

def 中华人民共和国农业农村部网站_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="中华人民共和国农业农村部网站"
    # 指定公司名称  
    page.get("http://www.moa.gov.cn")  
    # 输入公司名称并按回车键  
    page.ele("#loginname").input(f"{company_name}\n")
    page.wait.new_tab()
    # 切换到最新打开的标签页  
    page.to_tab(page.latest_tab)
    while True:
        ele2 = page.ele("仍然发送",timeout=0.3)
        if ele2:
            page.ele("仍然发送").click(None)
            continue
        if not ele2:
            break
    # 查找页面显示搜索信息页面的元素的大小，无内容为"符合条件的有0条"
    ele1 = page.ele("#results")
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if ele1.text == "":
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:  
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
        ele_text = ele1.text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")
    

def 中华人民共和国人力资源和社会保障部网站_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="中华人民共和国人力资源和社会保障部网站"
    # 指定公司名称  
    page.get("http://www.mohrss.gov.cn/")  
    # 输入公司名称并按回车键  
    page.ele("#loginname").input(f"{company_name}\n")
    page.wait.new_tab()
    # 切换到最新打开的标签页  
    page.to_tab(page.latest_tab)
    while True:
        ele2 = page.ele("仍然发送",timeout=0.3)
        if ele2:
            page.ele("仍然发送").click(None)
            continue
        if not ele2:
            break
    # 查找页面显示搜索信息页面的元素的大小，无内容为"符合条件的有0条"
    ele1 = page.ele("css:#otherwordidnew > div > div > div.ll_gjjs_list_mian")
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if len(ele1.text) == 37:
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:  
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
        ele_text = ele1.text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")
    
def 中华人民共和国住房和城乡建设部_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="中华人民共和国住房和城乡建设部"
    # 指定公司名称  
    page.get("https://www.mohurd.gov.cn/")
    # 输入公司名称并按回车键  
    page.ele("#searchInput").input(f"{company_name}\n")
    page.wait.new_tab()
    # 切换到最新打开的标签页  
    page.to_tab(page.latest_tab)
    while True:
        ele1 = page.ele("为您找到相关结果约")
        print(len((ele1.text)))
        if len(ele1.text) == 0:
            time.sleep(0.5)
            continue
        else:
            break
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if ele1.text[9:-1] == "0":
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:  
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
        ele_text = page.ele("css:#content > ul",timeout=5).text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")

def 国家统计局网站_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="国家统计局网站"
    # 指定公司名称  
    page.get("https://www.stats.gov.cn/")
    # 输入公司名称并按回车键  
    page.ele("#search").input(f"{company_name}\n")
    page.wait.new_tab()
    # 切换到最新打开的标签页  
    page.to_tab(page.latest_tab)
    while True:
        ele1 = page.ele(".noresult")
        print(len((ele1.text)))
        if len(ele1.text) == 0:
            time.sleep(0.5)
            continue
        else:
            break
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if len(ele1.text) == "":
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:  
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
        ele_text = page.ele("#results",timeout=5).text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")

def 国家市场监督管理总局网站_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="国家市场监督管理总局网站"
    # 指定公司名称  
    page.get("https://www.samr.gov.cn/")
    # 输入公司名称并按回车键  
    page.ele("#q").input(company_name)
    tabs1 = page.tabs_count
    while page.tabs_count==tabs1:
        page.ele(".search_btn").click(None)
    # 切换到最新打开的标签页  
    page.to_tab(page.latest_tab)
    while True:
        try:
            ele1 = page.ele("抱歉",timeout=3)#(728, 22)为已经搜到消息了且加载完了
            ele2 = page.ele(".content_cate",timeout=3)
            if ele2.size != (755,0) and len(ele2.text)>203:
                time.sleep(0.5)
                break
            elif ele1.size == (728, 22) and ele2.size == (755, 0):
                time.sleep(0.5)
                break
            else:
                time.sleep(0.5)
                continue
        except NoRectError:
            time.sleep(0.3)
            continue
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if ele2.size == (755, 0):
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:  
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
        ele_text = page.ele(".content_cate",timeout=5).text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")
    
def 国家药品监督管理局_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="国家药品监督管理局"
    # 指定公司名称  
    page.get("https://www.nmpa.gov.cn/index.html")  
    # 输入公司名称并按回车键  
    page.ele("#qt").input(company_name+"\n")
    # 切换到最新打开的标签页  
    page.to_tab(page.latest_tab)
    # 查找页面显示搜索信息页面的元素的大小，无内容为（1190，234）
    ele1 = page.ele("css:body > div.contentbox.box.clearfix")
    while True:
        if ele1.size[1] == 60:
            continue
        else:
            break
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if ele1.size[1] == 234:
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:  
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是234，就是有搜索内容，搜索的内容保存到df表中。
        ele_text = page.ele("#results").text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")
    
def 中国人民银行_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="中国人民银行"
    # 指定公司名称  
    page.get("http://www.pbc.gov.cn/")  
    # 输入公司名称并按回车键  
    page.ele("#q").input(company_name+"\n")
    # 切换到最新打开的标签页  
    page.to_tab(page.latest_tab)
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if page.ele("条结果").text[0] == "0":
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:  
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ele_text = page.ele(".default-result-list conMid_con").text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")
    
def 国家金融监督管理总局_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="国家金融监督管理总局"
    # 指定公司名称  
    page.get("https://www.cbirc.gov.cn/cn/view/pages/index/index.html")  
    # 输入公司名称并按回车键  
    page.ele("#search").input(company_name+"\n")
    # 切换到最新打开的标签页  
    page.to_tab(page.latest_tab)
    time.sleep(0.5)
    while True:
        if int(page.ele("共检索到").text[4])>=0:
            break
        else:
            continue
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if page.ele("共检索到").text[4] == "0":
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:  
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ele_text = page.ele(".jiansuo-right-result").text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")

def 国家外汇管理局网站_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="国家外汇管理局网站"
    # 指定公司名称  
    page.get("https://www.safe.gov.cn/")  
    # 输入公司名称并按回车键  
    page.ele("#title").input(company_name+"\n")
    # 切换到最新打开的标签页  
    page.to_tab(page.latest_tab)
    while True:
        if int(page.ele("条结果").text[2])>=0:
            break
        else:
            continue
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if page.ele("条结果").text[2] == "0":
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:  
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ele_text = page.ele(".sscontent").text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")
    
def 政府采购网_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="政府采购网"
    # 指定公司名称  
    page.get("http://www.ccgp.gov.cn/search/cr/")  
    # 输入公司名称并按回车键  
    page.ele(".gp-list-input").input(f"{company_name}\n")
    page.ele("@value=查找").click(None)
    # 查找页面显示搜索信息页面的元素的大小，无内容为"符合条件的有0条"
    ele1 = page.ele("没有该企业的相关记录")
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if ele1:
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:  
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
        ele_text = page.ele("#tableInfo").text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")
    
    
def 信用能源_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="信用能源"
    # 指定公司名称  
    page.get("http://xyny.nea.gov.cn/main")  
    # 输入公司名称并按回车键  
    page.ele("#globalContent").input(f"{company_name}\n")
    page.wait.new_tab()
    tab=page.get_tab(page.latest_tab)
    time.sleep(0.5)
    # 查找页面显示搜索信息页面的元素的大小，无内容为"符合条件的有0条"
    ele1 = tab.ele("未查询到相关企业")
    #???为什么不加这个ele1就不行，加了就行，不明白
    ele1
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if ele1.states.is_displayed:
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:
        ele_text = tab.ele(".search-table").text
        tab.ele("查看").click(None)
        tab=page.get_tab(page.latest_tab)
        tab.ele("报告查看时间")
        ad_to_df( company_name, url_name, "否", "是",ele_text)
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")
    
    
def 信用交通网_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="信用交通网"
    # 指定公司名称  
    page.get("https://credit.mot.gov.cn")  
    # 输入公司名称并按回车键  
    page.ele(".form-control").input(f"{company_name}\n")
    page.ele("搜索一下").click(None)
    page.wait.new_tab()
    tab=page.get_tab(page.latest_tab)
    # 查找页面显示搜索信息页面的元素的大小，无内容为"符合条件的有0条"
    ele1 = tab.ele("没有查询到相关信息！")
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if ele1:
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:
        ele_text = tab.ele(".table table-striped").text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")
    
def 中国商务信用平台_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="中国商务信用平台"
    # 指定公司名称  
    #page.get("https://www.bcpcn.com/")
    #https://www.bcpcn.com/bcp/dataq.jsp?qymc=1&pinpaimc=&zscode=
    # 输入公司名称并按回车键  
    page.get(f"https://www.bcpcn.com/bcp/dataq.jsp?qymc={company_name}&pinpaimc=&zscode=")
    #page.ele("搜索一下").click(None)
    #page.wait.new_tab()
    #tab=page.get_tab(page.latest_tab)
    # 查找页面显示搜索信息页面的元素的大小，无内容为"符合条件的有0条"
    ele1 = page.ele("查询无结果!")
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if ele1:
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:
        ele_text = page.ele(".inquire_list").text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")
    
    
def 信用盐业_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="信用盐业"
    # 指定公司名称  
    #page.get("https://www.bcpcn.com/")
    #https://www.bcpcn.com/bcp/dataq.jsp?qymc=1&pinpaimc=&zscode=
    # 输入公司名称并按回车键  
    page.get(f"http://yan.bcpcn.com/website/xyjl.jsp?keyword={company_name}&searchtype=1&page=1")
    
    #page.ele("搜索一下").click(None)
    #page.wait.new_tab()
    #tab=page.get_tab(page.latest_tab)
    # 查找页面显示搜索信息页面的元素的大小，无内容为"符合条件的有0条"
    ele1 = page.ele(".hmd gzmd l m_top")
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if ele1.text == '企业名称统一社会信用代码企业类型地区':
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:
        ele_text = ele1.text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")
    

def 信用电力_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="信用电力"
    # 指定公司名称  
    page.get("https://creditpower.cec.org.cn/")  
    # 输入公司名称并按回车键  
    page.ele("@placeholder=请输入关键词").input(f"{company_name}\n")
    #page.ele("搜索一下").click(None)
    page.wait.new_tab()
    tab=page.get_tab(page.latest_tab)
    # 查找页面显示搜索信息页面的元素的大小，无内容为"符合条件的有0条"
    ele1 = tab.ele(".news-list")
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if ele1.text == "":
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:
        ele_text = ele1.text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")

    
def 杭州市政府官方网站_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="杭州市政府官方网站"
    # 指定公司名称  
    #page.get("https://www.bcpcn.com/")
    #https://www.bcpcn.com/bcp/dataq.jsp?qymc=1&pinpaimc=&zscode=
    # 输入公司名称并按回车键  
    page.get(f"https://search.zj.gov.cn/jrobotfront/search.do?q={company_name}&websiteid=330101000000000&tpl=2296&p=1&Municipal_webid=330101000000000&Municipal_name=杭州市&isContains=0")
    
    #page.ele("搜索一下").click(None)
    #page.wait.new_tab()
    #tab=page.get_tab(page.latest_tab)
    # 查找页面显示搜索信息页面的元素的大小，无内容为"符合条件的有0条"
    ele1 = page.ele(".comprehensive")
    time.sleep(2)
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    ele_text = ele1.text
    ad_to_df( company_name, url_name, "否", "是",ele_text)
    save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
    #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")
    


def 信用中国网重点领域严重失信主体名单_严重失信主体名单查询_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="信用中国网重点领域严重失信主体名单_严重失信主体名单查询"
    page.get("https://www.creditchina.gov.cn/xinyongfuwu/shixinheimingdan/")  
    # 输入公司名称并按回车键  
    #@@name=row1@@class:cls
    page.ele(".searchBox").ele("@placeholder=请输入主体名称或者统一社会信用代码").input(f"{company_name}\n")
    time.sleep(0.3)
    # 查找页面显示搜索信息页面的元素的大小，无内容为"符合条件的有0条"
    ele1 = page.ele(".no-data-message")
    while True:
        if ele1.states.is_displayed:
            break
        else:
            continue
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if ele1:
        page.scroll.to_bottom()
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:
        ele_text = tab.ele(".no-data-message").text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")
    
def 信用中国网重点领域严重失信主体名单_经营异常名录信息查询_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="信用中国网重点领域严重失信主体名单_经营异常名录信息查询"
    page.get("https://www.creditchina.gov.cn/xinyongfuwu/zhongdianguanzhushixinmingdan/")  
    # 输入公司名称并按回车键  
    #@@name=row1@@class:cls
    page.ele(".searchBox").ele("@placeholder=请输入主体名称或者统一社会信用代码").input(f"{company_name}\n")
    time.sleep(0.3)
    # 查找页面显示搜索信息页面的元素的大小，无内容为"符合条件的有0条"
    ele1 = page.ele(".no-data-message")
    while True:
        if ele1.states.is_displayed:
            break
        else:
            continue
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if ele1:
        page.scroll.to_bottom()
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:
        ele_text = tab.ele(".no-data-message").text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")
    
def 信用中国网重点领域严重失信主体名单_政府采购严重违法失信行为记录名单_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="政府采购严重违法失信行为记录名单"
    page.get("https://www.creditchina.gov.cn/xinyongfuwu/zhengfucaigouyanzhongweifashixinmingdan/")  
    # 输入公司名称并按回车键  
    #@@name=row1@@class:cls
    page.ele(".searchBox").ele("@placeholder=请输入主体名称或者统一社会信用代码").input(f"{company_name}\n")
    time.sleep(0.3)
    # 查找页面显示搜索信息页面的元素的大小，无内容为"符合条件的有0条"
    ele1 = page.ele(".no-data-message")
    while True:
        if ele1.states.is_displayed:
            break
        else:
            continue
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if ele1:
        page.scroll.to_bottom()
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:
        ele_text = tab.ele(".no-data-message").text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")
    
def 信用中国网重点领域严重失信主体名单_拖欠农民工工资失信联合惩戒对象名单_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="信用中国网重点领域严重失信主体名单_拖欠农民工工资失信联合惩戒对象名单"
    page.get("https://www.creditchina.gov.cn/xinyongfuwu/zdlycx/?tablename=credit_rsb_fr_tqnmggz&gsName=%E6%8B%96%E6%AC%A0%E5%86%9C%E6%B0%91%E5%B7%A5%E5%B7%A5%E8%B5%84%E5%A4%B1%E4%BF%A1%E8%81%94%E5%90%88%E6%83%A9%E6%88%92%E5%AF%B9%E8%B1%A1%E5%90%8D%E5%8D%95")  
    # 输入公司名称并按回车键  
    #@@name=row1@@class:cls
    page.ele(".searchBox").ele("@placeholder=请输入主体名称").input(f"{company_name}\n")
    time.sleep(0.3)
    # 查找页面显示搜索信息页面的元素的大小，无内容为"符合条件的有0条"
    ele1 = page.ele(".no-data-message")
    while True:
        if ele1.states.is_displayed:
            break
        else:
            continue
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if ele1:
        page.scroll.to_bottom()
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:
        ele_text = tab.ele(".no-data-message").text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")
    
def 信用中国网重点领域严重失信主体名单_重大税收违法失信主体_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="信用中国网重点领域严重失信主体名单_重大税收违法失信主体"
    page.get("https://www.creditchina.gov.cn/xinyongfuwu/zhongdashuishouweifaanjian/")  
    # 输入公司名称并按回车键  
    #@@name=row1@@class:cls
    page.ele(".searchBox").ele("@placeholder=请输入主体名称或者统一社会信用代码").input(f"{company_name}\n")
    time.sleep(0.3)
    # 查找页面显示搜索信息页面的元素的大小，无内容为"符合条件的有0条"
    ele1 = page.ele(".no-data-message")
    while True:
        if ele1.states.is_displayed:
            break
        else:
            continue
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if ele1:
        page.scroll.to_bottom()
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:
        ele_text = tab.ele(".no-data-message").text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")
    
def 信用中国网重点领域严重失信主体名单_统计严重失信企业名单_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="信用中国网重点领域严重失信主体名单_统计严重失信企业名单"
    page.get("https://www.creditchina.gov.cn/xinyongfuwu/zdlycx/?tablename=credit_gjtjj_fr_tjsyzsxqy&gsName=%E7%BB%9F%E8%AE%A1%E4%B8%A5%E9%87%8D%E5%A4%B1%E4%BF%A1%E4%BC%81%E4%B8%9A%E5%90%8D%E5%8D%95")  
    # 输入公司名称并按回车键  
    #@@name=row1@@class:cls
    page.ele(".searchBox").ele("@placeholder=请输入主体名称或者统一社会信用代码").input(f"{company_name}\n")
    time.sleep(0.3)
    # 查找页面显示搜索信息页面的元素的大小，无内容为"符合条件的有0条"
    ele1 = page.ele(".no-data-message")
    while True:
        if ele1.states.is_displayed:
            break
        else:
            continue
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if ele1:
        page.scroll.to_bottom()
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:
        ele_text = tab.ele(".no-data-message").text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")
    
    
def 重大税收违法失信案件信息公布栏_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="重大税收违法失信案件信息公布栏"
    page.get("https://zhejiang.chinatax.gov.cn/col/col24411/index.html")  
    # 输入公司名称并按回车键  
    #@@name=row1@@class:cls
    page.ele(".input1").input(f"{company_name}\n")
    page.ele(".search-submit").click(None)
    time.sleep(0.3)
    # 查找页面显示搜索信息页面的元素的大小，无内容为"符合条件的有0条"
    ele1 = page.ele("很遗憾，没有检索到任何记录！")
    while True:
        if ele1.states.is_displayed:
            break
        else:
            continue
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    if ele1:
        #page.scroll.to_bottom()
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:
        ele_text = tab.ele(".no-data-message").text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")

In [5]:
def 中国执行信息公开网_main(company_name):
    """
    company_name：输入的公司名称
    """
    while True:
        url_name="中国执行信息公开网"
        page.get("http://zxgk.court.gov.cn/shixin/")  
        # 输入公司名称并按回车键  
        page.ele("#pName").input(company_name)
        page("#yzm").click(None)
        time.sleep(0.5)
        img = page("#captchaImg")
        src = img.get_src()  
        # 保存采集的img图片  
        img.save("img","src.jpg")  
        # 识别验证码  
        text_验证码 = dd_验证码识别_pt()  
        # 寻找验证码输入文本框，并输入识别的text_验证码  
        #page.click("#yzm").type(text_验证码)
        page("#yzm").input(text_验证码)
        # 点击搜索按钮 
        page.ele(".btn btn-zxgk btn-block ").click(None)
        print("正在校验，内容是否详实"+company_name)
        time.sleep(2)
        print(page("xpath://html/body/div[2]/div/div[2]/div[4]/div[1]/div/table/tbody/p").text)
        if page("xpath://html/body/div[2]/div/div[2]/div[4]/div[1]/div/table/tbody/p").text == "在全国范围内没有找到"+company_name+"相关的结果.":
            time.sleep(2)
            save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
            ad_to_df(company_name,url_name, "否", "是","")
            print("true，执行公开查询完成")
            break
        elif page("xpath://html/body/div[2]/div/div[2]/div[4]/div[1]/div/table/tbody/p").text == "验证码错误或验证码已过期。":
            print("验证码错误，重新开始"+company_name)
            continue
        else:
            print("其他错误")
            print(page("xpath://html/body/div[2]/div/div[2]/div[4]/div[1]/div/table/tbody/p").text)
        #判断是否找到了失信惩戒对象，并打印相应的消息。
        page.close_other_tabs()
        print(f"【{company_name}】已经完成【{url_name}】的核查")

In [6]:
def 证券期货市场失信记录查询平台_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="证券期货市场失信记录查询平台"
    page.get("https://neris.csrc.gov.cn/shixinchaxun/")  
    # 输入公司名称并按回车键  
    page.ele("@placeholder=必须填写姓名或名称，至少两个以上汉字").input(f"{company_name}\n")
    page("搜索").click(None)
    time.sleep(1)
    #验证码模块
    big_pic_path=r"img/big_pic.png"
    time.sleep(3)
    page("css:#app > div > div > div.mask > div > div.verifybox-bottom > div > div.verify-img-out > div > img").save(None,big_pic_path)
    small_pic_path = r"img/small_pic.png"
    page("css:#app > div > div > div.mask > div > div.verifybox-bottom > div > div.verify-bar-area > div > div > div > img").save(None,small_pic_path)
    res=ddddocr_滑块(big_pic_path,small_pic_path)
    ac = ActionChains(page)
    ele=("css:#app > div > div > div.mask > div > div.verifybox-bottom > div > div.verify-bar-area > div > div")
    ac.hold(ele).right(int(res))
    time.sleep(0.5)
    ac.left(25)
    time.sleep(0.3)
    ac.release()
    time.sleep(10)
    page.to_tab(page.latest_tab)
    save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
    ad_to_df(company_name,url_name, "否", "是","")
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")

In [7]:
def 全国行业信用公共服务平台_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="全国行业信用公共服务平台"
    page.get("http://www.bcp12312.org.cn/")
    page.ele("@name=kd").input(f"{company_name}\n")
    #验证码模块
    page("#licenceImg").save("img","src.jpg")
    page("#validatecode").input(dd_验证码识别_pt()+"\n")
    a=page.handle_alert(timeout=1)
    time.sleep(1)        
    if  a == False:
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")

In [8]:
def 慧眼大数据风险监测系统_main(company_name):
    """
    company_name：输入的公司名称
    """
    url_name="慧眼大数据风险监测系统"
    page.get("http://222.73.4.143:8088/index")
    if page.url == "http://222.73.4.143:8088/loginPage":
        page.ele("@placeholder=用户名/手机号").input("htbond01")
        page.ele("@placeholder=密码").input("htbond01")
        page.ele(".loginButton").click(None)
    page.ele("查诚信").click(None)
    page.ele("@placeholder=输入公司或者证券").input(company_name)
    time.sleep(1)
    page.ele("@placeholder=输入公司或者证券").input("\n")
    time.sleep(1)
    page.ele("@placeholder=输入公司或者证券").input("\n")
    page.ele("@class=banbutt").click(None)
    time.sleep(5)
    # 查找页面显示搜索信息页面的元素的大小，无内容为"符合条件的有0条"
    ele1 = page.ele("沪深上市")
    while True:
        if ele1.states.is_displayed:
            break
        else:
            time.sleep(0.1)
            continue
    #判断是否找到了失信惩戒对象，并打印相应的消息。
    time.sleep(1)
    if page.ele("@class=grid-container").text == "":
        page.scroll.to_bottom()
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:
        ele_text = page.ele("@class=grid-container").text
        ad_to_df( company_name, url_name, "否", "是",ele_text)
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        #反之不是0，就是有搜索内容，搜索的内容保存到df表中。
    page.close_other_tabs()
    print(f"【{company_name}】已经完成【{url_name}】的核查")

In [18]:
def 中国海关企业进出口信用信息公示平台_main(company_name):  
    """
    company_name：输入的公司名称
    """
    while True:
        url_name="中国海关企业进出口信用信息公示平台"
        page.get("http://credit.customs.gov.cn/")
        page.ele("@value=企业名称或统一社会信用代码").input(f"{company_name}\n")
        #验证码模块
        img = page("#verifyCode")  
        src = img.get_src()
        img.save('img',"src.gif")
        text_验证码 = dd_验证码识别_gif("img/src.gif")  
        # 寻找验证码输入文本框，并输入识别的text_验证码  
        page("#checkCode").input(text_验证码)
        for i in range(5):
            try:
                time.sleep(3)
                page("xpath://html/body/div/div[3]/div[1]/div[1]/button").click()
                break
            except ElementNotFoundError:
                time.sleep(3)
                page("xpath://html/body/div/div[3]/div[1]/div[1]/button").click()
        print("正在校验，内容是否详实")
            # 查找提示文本框，校验内容为“无符合条件的数据”或“验证码错误”。
        if page.ele("xpath://html/body/div[3]/div[2]",timeout=5):
            if page('xpath://html/body/div[3]/div[2]').text == "无符合条件的数据":  
                save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
                ad_to_df(company_name,url_name, "否", "是","")
                print("海关完成true")
                break  
            elif page('xpath://html/body/div[3]/div[2]').text == "请输入正确的验证码":  
                print("验证码错误，重新开始运行")
                continue  # 当出现验证码错误时，重新开始循环  
            else:  
                print(company_name+"出现其他问题") 
                #continue
                continue
        else:
            page.refresh()
            time.sleep(1)
            continue

In [59]:
def 国家企业信用信息公示系统_main(company_name):  
    """
    company_name：输入的公司名称
    """
    url_name="国家企业信用信息公示系统"
    #page.get("http://www.gsxt.gov.cn")
    page.get("https://shiming.gsxt.gov.cn/")
    page.ele("@placeholder=请输入企业名称、统一社会信用代码或注册号",timeout=100).input(f"{company_name}\n")
    #验证码模块（等待用户输入）
    if "查询到0条信息" in page.ele("查询",timeout=10000).text:
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:
        page.ele(company_name).click(None)
        page.wait.new_tab()
        page.to_tab(page.latest_tab)
        page.ele(".menu").ele("基础信息").click(None)
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
        国家企业信用信息公示系统_items(company_name, url_name)
        

def 国家企业信用信息公示系统_items(company_name, url_name):
    
    """  
    处理多个菜单项的函数。  
  
    参数:  
    - company_name: 字符串，公司名称，用于构造截图和数据框的保存路径。  
    - url_name: 字符串，URL名称或标识符，同样用于构造截图和数据框的保存路径。
    内部参数:
    - menu_items: 列表，包含多个元组，每个元组表示一个菜单项的信息。  
                 每个元组包含三个元素：  
                 1. - menu_name: 菜单名称（字符串），用于定位并点击页面上的菜单项。  
                 2. - menu_id: 元素ID（字符串），用于定位页面上用于获取信息的元素。
                 3. - menu_Lnub: 元素文本的长度（数字），用于判断是否加载完成 
  
    返回值:  
    - None: 此函数不返回任何值，但会执行一系列操作，包括点击菜单项、打印信息、保存截图和添加到数据框。  
    """
    
    menu_items = [("行政许可信息","nLictb",39),
                ("行政处罚信息","needPaging_punish",44),
                ("列入经营异常名录信息","needPaging_abnormal",56),
                ("列入严重违法失信名单（黑名单）信息","needPaging_illegal",73),
                ("公告信息", "stopBusi_table",22)]
    for menu_item in menu_items:  
        menu_name, menu_id, menu_Lnub = menu_item  

        # 点击菜单项  
        page.ele(".menu").ele(menu_name).click(None)  

        # 获取并打印相关信息  
        while True:
            info_text = page.ele(f"#{menu_id}").text 
            if len(info_text) != menu_Lnub:
                #print(info_text)  
                # 保存截图  
                screenshot_path = f"data/{company_name}/{company_name}_{url_name}_{menu_name}.jpg"    
                save_screenshot_to_custom_path(screenshot_path)
                # 添加到数据框  
                ad_to_df(company_name, url_name, "否", "是", info_text)
                break
            else:
                time.sleep(0.3)
                continue

In [69]:
def 中国裁判文书网_main(company_name):  
    """
    company_name：输入的公司名称
    """
    url_name="中国裁判文书网"
    page.get("https://wenshu.court.gov.cn/")
    page.ele("欢迎您")
    page.ele("@placeholder=输入案由、关键词、法院、当事人、律师",timeout=100).input(f"{company_name}\n")
    #验证码模块（等待用户输入）
    ele1 = page.ele("共检索到",timeout=10000)
    if "共检索到0篇文书" in ele1.text:
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是","")
    else:
        save_screenshot_to_custom_path(f"data/{company_name}/{company_name}_{url_name}.jpg")
        ad_to_df(company_name,url_name, "否", "是",ele1.text)

In [10]:
def 核查意见_ad_是否适用_hc(company_name,url_name):
    try:
        a = df[(df['公司名称'] == company_name) & (df['网址名称'] == url_name)]['核查意见'].values[0]
        b = df[(df['公司名称'] == company_name) & (df['网址名称'] == url_name)]['是否适用'].values[0]
    except IndexError:
        a = "否"
        b = "是"
    return a,b

In [67]:
company_name_list=["财通证券（香港）有限公司",
"财通证券资产管理有限公司",
"财通证券股份有限公司",
"财通国际资产管理有限公司",
"财通国际证券有限责任公司"]
文件创建(company_name_list)

文件夹已存在: 财通证券（香港）有限公司
文件夹已存在: 财通证券资产管理有限公司
文件夹已存在: 财通证券股份有限公司
文件夹已存在: 财通国际资产管理有限公司
文件夹已存在: 财通国际证券有限责任公司


In [12]:
#df.to_excel('data.xlsx', index=False)

In [18]:
page = ChromiumPage()
for company_name in company_name_list:
    中华人民共和国国家发展和改革委员会网站_main(company_name)
    中华人民共和国应急管理部网站_main(company_name)
    中华人民共和国生态环境部网站_main(company_name)
    中华人民共和国工业和信息化部网站_main(company_name)
    中华人民共和国商务部网站_main(company_name)
    中华人民共和国财政部网站_main(company_name)
    中华人民共和国农业农村部网站_main(company_name)
    中华人民共和国人力资源和社会保障部网站_main(company_name)
    中华人民共和国住房和城乡建设部_main(company_name)
    国家统计局网站_main(company_name)
    国家市场监督管理总局网站_main(company_name)
    国家药品监督管理局_main(company_name)
    中国人民银行_main(company_name)
    国家金融监督管理总局_main(company_name)
    国家外汇管理局网站_main(company_name)
    政府采购网_main(company_name)
    信用能源_main(company_name)
    信用交通网_main(company_name)
    中国商务信用平台_main(company_name)
    信用盐业_main(company_name)
    信用电力_main(company_name)
    杭州市政府官方网站_main(company_name)
    信用中国网重点领域严重失信主体名单_严重失信主体名单查询_main(company_name)
    信用中国网重点领域严重失信主体名单_经营异常名录信息查询_main(company_name)
    信用中国网重点领域严重失信主体名单_政府采购严重违法失信行为记录名单_main(company_name)
    信用中国网重点领域严重失信主体名单_拖欠农民工工资失信联合惩戒对象名单_main(company_name)
    信用中国网重点领域严重失信主体名单_重大税收违法失信主体_main(company_name)
    信用中国网重点领域严重失信主体名单_统计严重失信企业名单_main(company_name)
df

Exception in thread Thread-150:
Traceback (most recent call last):
  File "d:\program files\python\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "d:\program files\python\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "d:\program files\python\lib\site-packages\DrissionPage\chromium_driver.py", line 123, in _recv_loop
    message = loads(message_json)
  File "d:\program files\python\lib\json\__init__.py", line 357, in loads
    return _default_decoder.decode(s)
  File "d:\program files\python\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "d:\program files\python\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


【财通证券股份有限公司】已经完成【中华人民共和国国家发展和改革委员会网站】的核查


Exception in thread Thread-154:
Traceback (most recent call last):
  File "d:\program files\python\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "d:\program files\python\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "d:\program files\python\lib\site-packages\DrissionPage\chromium_driver.py", line 123, in _recv_loop
    message = loads(message_json)
  File "d:\program files\python\lib\json\__init__.py", line 357, in loads
    return _default_decoder.decode(s)
  File "d:\program files\python\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "d:\program files\python\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


【财通证券股份有限公司】已经完成【中华人民共和国应急管理部网站】的核查


Exception in thread Thread-156:
Traceback (most recent call last):
  File "d:\program files\python\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "d:\program files\python\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "d:\program files\python\lib\site-packages\DrissionPage\chromium_driver.py", line 123, in _recv_loop
    message = loads(message_json)
  File "d:\program files\python\lib\json\__init__.py", line 357, in loads
    return _default_decoder.decode(s)
  File "d:\program files\python\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "d:\program files\python\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


...

Exception in thread Thread-558:
Traceback (most recent call last):
  File "d:\program files\python\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "d:\program files\python\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "d:\program files\python\lib\site-packages\DrissionPage\chromium_driver.py", line 123, in _recv_loop
    message = loads(message_json)
  File "d:\program files\python\lib\json\__init__.py", line 357, in loads
    return _default_decoder.decode(s)
  File "d:\program files\python\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "d:\program files\python\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


【浙江股权服务集团有限公司】已经完成【国家药品监督管理局】的核查


Exception in thread Thread-560:
Traceback (most recent call last):
  File "d:\program files\python\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "d:\program files\python\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "d:\program files\python\lib\site-packages\DrissionPage\chromium_driver.py", line 123, in _recv_loop
    message = loads(message_json)
  File "d:\program files\python\lib\json\__init__.py", line 357, in loads
    return _default_decoder.decode(s)
  File "d:\program files\python\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "d:\program files\python\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


【浙江股权服务集团有限公司】已经完成【中国人民银行】的核查
【浙江股权服务集团有限公司】已经完成【国家金融监督管理总局】的核查
【浙江股权服务集团有限公司】已经完成【国家外汇管理局网站】的核查
【浙江股权服务集团有限公司】已经完成【政府采购网】的核查
【浙江股权服务集团有限公司】已经完成【信用能源】的核查
【浙江股权服务集团有限公司】已经完成【信用交通网】的核查
【浙江股权服务集团有限公司】已经完成【中国商务信用平台】的核查
【浙江股权服务集团有限公司】已经完成【信用盐业】的核查
【浙江股权服务集团有限公司】已经完成【信用电力】的核查
【浙江股权服务集团有限公司】已经完成【杭州市政府官方网站】的核查
【浙江股权服务集团有限公司】已经完成【信用中国网重点领域严重失信主体名单_严重失信主体名单查询】的核查
【浙江股权服务集团有限公司】已经完成【信用中国网重点领域严重失信主体名单_经营异常名录信息查询】的核查
【浙江股权服务集团有限公司】已经完成【政府采购严重违法失信行为记录名单】的核查
【浙江股权服务集团有限公司】已经完成【信用中国网重点领域严重失信主体名单_拖欠农民工工资失信联合惩戒对象名单】的核查
【浙江股权服务集团有限公司】已经完成【信用中国网重点领域严重失信主体名单_重大税收违法失信主体】的核查
【浙江股权服务集团有限公司】已经完成【信用中国网重点领域严重失信主体名单_统计严重失信企业名单】的核查


,公司名称,网址名称,核查意见,是否适用,其他数据
0,香港中央结算（代理人）有限公司,中华人民共和国国家发展和改革委员会网站,否,是,
1,香港中央结算（代理人）有限公司,中华人民共和国应急管理部网站,否,是,
2,香港中央结算（代理人）有限公司,中华人民共和国生态环境部网站,否,是,法规标准沪港股票市场交易互联互通机制试点若干规定\n2014-06-01 \t\t\t\t\...
3,香港中央结算（代理人）有限公司,中华人民共和国工业和信息化部网站,否,是,
4,香港中央结算（代理人）有限公司,中华人民共和国商务部网站,否,是,
...,...,...,...,...,...
461,浙江股权服务集团有限公司,信用中国网重点领域严重失信主体名单_经营异常名录信息查询,否,是,
462,浙江股权服务集团有限公司,政府采购严重违法失信行为记录名单,否,是,
463,浙江股权服务集团有限公司,信用中国网重点领域严重失信主体名单_拖欠农民工工资失信联合惩戒对象名单,否,是,
464,浙江股权服务集团有限公司,信用中国网重点领域严重失信主体名单_重大税收违法失信主体,否,是,


In [70]:
page = ChromiumPage()
for company_name in company_name_list:
    #中国执行信息公开网_main(company_name)
    #重大税收违法失信案件信息公布栏_main(company_name)
    #证券期货市场失信记录查询平台_main(company_name)
    #全国行业信用公共服务平台_main(company_name)
    #慧眼大数据风险监测系统_main(company_name)
    #中国海关企业进出口信用信息公示平台_main(company_name)
    #国家企业信用信息公示系统_main(company_name)
    中国裁判文书网_main(company_name)
df

,公司名称,网址名称,核查意见,是否适用,其他数据
0,财通证券（香港）有限公司,中国海关企业进出口信用信息公示平台,否,是,
1,财通证券资产管理有限公司,中国海关企业进出口信用信息公示平台,否,是,
2,财通证券（香港）有限公司,中国海关企业进出口信用信息公示平台,否,是,
3,财通证券资产管理有限公司,中国海关企业进出口信用信息公示平台,否,是,
4,财通证券股份有限公司,中国海关企业进出口信用信息公示平台,否,是,
5,财通国际资产管理有限公司,中国海关企业进出口信用信息公示平台,否,是,
6,财通国际证券有限责任公司,中国海关企业进出口信用信息公示平台,否,是,
7,财通国际融资有限公司,中国海关企业进出口信用信息公示平台,否,是,
8,财通国际投资有限公司,中国海关企业进出口信用信息公示平台,否,是,
9,财通创新投资有限公司,中国海关企业进出口信用信息公示平台,否,是,
